### Import packages and set up paths

In [ ]:
import os
import re
import socket
import numpy as np
import pandas as pd

import scipy.stats as stats
from scipy.spatial import distance as dist
from scipy.spatial.distance import pdist
from sklearn import preprocessing
from scipy.stats import f

import matplotlib
from matplotlib import pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')


from utils import feature_heatmap, compute_f_stat, between_condition_RDM, photoid_sd_barplots, photoid_sd_distplots, generate_acc_probs
from classdata import Data

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
if socket.gethostname() == 'nightingale':
    feature_dir = os.path.abspath('/mnt/pentagon/photodraw/features/')
else:
    feature_dir = os.path.abspath(os.path.join(proj_dir,'features', 'photodraw12'))
    
image_path = os.path.abspath(os.path.join(feature_dir, 'flattened_sketches_pixels.npy'))
image_path_fc6 = os.path.abspath(os.path.join(feature_dir, 'FEATURES_FC6_photodraw_sketch.npy'))

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir,csv_dir,feature_dir]]

### Read in data, create preprocessing functions

In [ ]:
# read in data
T = pd.read_csv(os.path.join(csv_dir,'photodraw_stroke_data.csv'))
K = pd.read_csv(os.path.join(csv_dir,'photodraw_sketch_data.csv'))
S = pd.read_csv(os.path.join(csv_dir,'photodraw_survey_data.csv'))
F = np.load(image_path)
F_fc6 = np.load(image_path_fc6)

In [ ]:
# remove images flagged as invalid or outliers
def remove_invalid(frame):
    return frame[frame.isInvalid == False]
def remove_flagged(frame):
    return frame[(frame.isOutlier == False) & (frame.isInvalid==False)]
def remove_invalid_T(T):
    thinghthing = K[K.isInvalid==True][['gameID','trialNum']].values
    return T[(~T.gameID.isin(thinghthing[:][0])) & (~T.trialNum.isin(thinghthing[:][1]))]
def remove_flagged_T(T):
    thinghthing = K[(K.isOutlier==True) | (K.isInvalid==True)][['gameID','trialNum']].values
    return T[(~T.gameID.isin(thinghthing[:][0])) & (~T.trialNum.isin(thinghthing[:][1]))]

In [ ]:
## do we really want to remove invalid trials and outlier trials?
remove_flagged = {'invalid' : True,
                  'outlier' : True}
normalize_mean = True
normalize_std = False

data = Data(K, F, F_fc6)

data.filter_out(remove_flagged)
# perform sanity checks to see if 
Ksanity = pd.read_csv(os.path.join(csv_dir,'photodraw_sketch_data.csv'))
assert np.sum(data.metadata.isInvalid) == 0 if remove_flagged['invalid'] else np.sum(data.metadata.isInvalid) == np.sum(Ksanity.isInvalid)
assert np.sum(data.metadata.isOutlier) == 0 if remove_flagged['outlier'] else np.sum(data.metadata.isOutlier) == np.sum(Ksanity.isOutlier)

data.preprocess(mean = normalize_mean, std = normalize_std)

## Analyses I: Variation-based observations

### Comparing within-category variation across conditions  

In [ ]:
photoid_sd_barplots(data.metadata, 'totalInk', 'total ink')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5));
# 1, 2, and 3 represent the three photo stimuli

In [ ]:
photoid_sd_barplots(data.metadata, 'activeSketchTime', 'active sketch time')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5));

In [ ]:
photoid_sd_barplots(data.metadata, 'numStrokes', 'number of strokes')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5));

In [ ]:
photoid_sd_distplots(data.metadata,'totalInk', 'total ink used')

In [ ]:
photoid_sd_distplots(data.metadata,'activeSketchTime', 'active sketch time')

In [ ]:
photoid_sd_distplots(data.metadata,'numStrokes', 'number of strokes')

In [ ]:
# condfeatures: makes two lists of the pixel-level features of each condition (photo, text)
condfeatures = [data.pixels[data.metadata[data.metadata['condition']==condition].index] for condition in ['photo','text']]
# compute pairwise distances across the pixels of all sketches in the two cues
photo_sims = pdist(condfeatures[0])
text_sims = pdist(condfeatures[1])

sns.distplot(photo_sims, hist=False,label='photo')
sns.distplot(text_sims, hist=False,label='text')
plt.xlabel('euclidean distances') 
plt.ylabel('density')
plt.title('Distibution of pariwise distances over category')
plt.legend()
plt.show()

In [ ]:
# get dictionary that maps category-photoid pairs to their respective set of indices in F and M 
indexmap = data.metadata.groupby(['category','photoid']).apply(lambda x: x.index.tolist()).to_dict()

# get mean and standard deviation of pairwise distance sketches in each category-photoid pair, also mean pixel features
pixel_dists_frame = pd.DataFrame(columns=['category','photoid','cue_id','mean_dist','std_dist','mean_feature'])
for index,key in enumerate(indexmap.keys()):
    pixel_dists_frame.loc[index] = [key[0], 
                                    key[1], 
                                    key[0] + '_' + key[1],
                                    pdist(data.pixels[indexmap[key]]).mean(), 
                                    pdist(data.pixels[indexmap[key]]).std(), 
                                    data.pixels[indexmap[key]].mean(axis=0)]
    
# adding condition saves headaches down the line
pixel_dists_frame = pixel_dists_frame.assign(condition = (['photo']*3 + ['text']) * 12)

# get dictionary that maps category-photoid pairs to their respective set of indices in F and M for fc6 features 
indexmap_fc6 = data.metadata.groupby(['category','photoid']).apply(lambda x: x.index.tolist()).to_dict()

# get mean and standard deviation of pairwise distance sketches in each category-photoid pair, also mean fc6 features
pixel_dists_frame_fc6 = pd.DataFrame(columns=['category','photoid','cue_id','mean_dist','std_dist','mean_feature'])
for index,key in enumerate(indexmap_fc6.keys()):
    pixel_dists_frame_fc6.loc[index] = [key[0],
                                        key[1],
                                        key[0] + '_' + key[1],
                                        pdist(data.fc6[indexmap_fc6[key]]).mean(),
                                        pdist(data.fc6[indexmap_fc6[key]]).std(), 
                                        data.fc6[indexmap_fc6[key]].mean(axis=0)]
    
# adding condition saves headaches down the line
pixel_dists_frame_fc6 = pixel_dists_frame_fc6.assign(condition = (['photo']*3 + ['text']) * 12)

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(pixel_dists_frame[pixel_dists_frame.condition == 'photo'].mean_dist.values, label = 'photo')
sns.distplot(pixel_dists_frame[pixel_dists_frame.condition == 'text'].mean_dist.values, label = 'text')
plt.xlabel('mean euclidean distance'), plt.ylabel('density'), plt.title('Average pairwise equclidean distance of photo-id vs text sketches').set_position([.5, 1.05])
plt.legend();

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(pixel_dists_frame[pixel_dists_frame.condition == 'photo'].std_dist.values, label = 'photo')
sns.distplot(pixel_dists_frame[pixel_dists_frame.condition == 'text'].std_dist.values, label = 'text')
plt.xlabel('standard deviation pairwise euclidean distance'), plt.ylabel('density'),plt.title('Standard deviation of pairwise equclidean distance of photo-id vs text sketches').set_position([.5, 1.05])
plt.legend();

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(data=pixel_dists_frame, x='category',y='mean_dist',hue='photoid')
plt.ylabel('mean pairwise distance'), plt.title('Mean pairwise euclidean distance within each cue-category pair');
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5));

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(data=pixel_dists_frame, x='category',y='std_dist',hue='photoid')
plt.ylabel('standard deviation pairwise distance'), plt.title('Std. dev of pairwise euclidean distance within each cue-category pair');
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5));

In [ ]:
plt.figure(figsize=(9,6))
sns.barplot(data=pixel_dists_frame, x='condition',y='std_dist')
plt.ylabel('standard deviation pairwise distance'), plt.title('Standard deviation pairwise euclidean distance across condition').set_position([.5, 1.05]);

In [ ]:
plt.figure(figsize=(9,6))
sns.barplot(data=pixel_dists_frame, x='condition',y='mean_dist')
plt.ylabel('mean pairwise distance'), plt.title('Mean pairwise euclidean distance across condition').set_position([.5, 1.05]);

In [ ]:
# condfeatures: makes two lists of the pixel-level features of each condition (photo, text)
condfeatures_fc6 = [data.fc6[data.metadata[data.metadata['condition'] == condition].index] for condition in ['photo','text']]

# compute pairwise distances across the pixels of all sketches in the two cues
photo_sims_fc6 = pdist(condfeatures_fc6[0])
text_sims_fc6 = pdist(condfeatures_fc6[1])

plt.figure(figsize=(8,6))
sns.distplot(photo_sims_fc6, hist=False, label='photo')
sns.distplot(text_sims_fc6, hist=False, label='text')
plt.xlabel('euclidean distances'), plt.ylabel('density'), plt.title('Distibution of pairwise distances over category').set_position([.5, 1.05])
plt.legend();

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(pixel_dists_frame_fc6[pixel_dists_frame_fc6.photoid != 'text'].mean_dist.values, label = 'photo')
sns.distplot(pixel_dists_frame_fc6[pixel_dists_frame_fc6.photoid == 'text'].mean_dist.values, label = 'text')
plt.xlabel('mean euclidean distance'), plt.ylabel('density'), plt.title('Average pairwise euclidean distance of photo-id vs text sketches').set_position([.5, 1.05])
plt.legend();

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(pixel_dists_frame_fc6[pixel_dists_frame_fc6.photoid != 'text'].std_dist.values, label = 'photo')
sns.distplot(pixel_dists_frame_fc6[pixel_dists_frame_fc6.photoid == 'text'].std_dist.values, label = 'text')
plt.xlabel('standard deviation pairwise euclidean distance'), plt.ylabel('density'), plt.title('Standard deviation of pairwise euclidean distance of photo-id vs text sketches')
plt.legend();

In [ ]:
plt.figure(figsize=(9,6))
sns.barplot(data=pixel_dists_frame_fc6, x='condition',y='std_dist')
plt.ylabel('std. pairwise distance'), plt.title('Standard deviation pairwise euclidean distance across condition').set_position([.5, 1.05]);

In [ ]:
plt.figure(figsize=(9,6))
sns.barplot(data=pixel_dists_frame_fc6, x='condition',y='mean_dist')
plt.ylabel('mean pairwise distance'), plt.title('Mean pairwise euclidean distance across condition').set_position([.5, 1.05]);

### Heatmaps for pixel-wise euclidean distance

In [ ]:
plt.figure(figsize=(12,14))
sns.barplot('mean_dist','cue_id', data=pixel_dists_frame)
plt.xlabel('Mean within-class euclidean distance'), plt.ylabel('Cue id'), plt.title('Mean within-class euclidean distance of each cue id (pixel-wise)');

In [ ]:
feature_heatmap(pixel_dists_frame, abstraction = 'pixel-level', metric='euclidean')

In [ ]:
feature_heatmap(pixel_dists_frame, abstraction = 'pixel-level', metric='cosine')

In [ ]:
feature_heatmap(pixel_dists_frame, abstraction = 'pixel-level', metric='correlation')

### Heatmaps for feature-wise euclidean distance

In [ ]:
plt.figure(figsize=(12,14))
sns.barplot('mean_dist','cue_id', data=pixel_dists_frame_fc6)
plt.xlabel('Mean within-class euclidean distance'), plt.ylabel('cue id'), plt.title('Mean within-class euclidean distance of each cue id (feature-wise)');

In [ ]:
feature_heatmap(pixel_dists_frame_fc6, abstraction = 'feature-level', metric='euclidean')

In [ ]:
feature_heatmap(pixel_dists_frame_fc6, abstraction = 'feature-level', metric='cosine')

In [ ]:
feature_heatmap(pixel_dists_frame_fc6, abstraction = 'feature-level', metric='correlation')

### F statistic analysis

In [ ]:
# compute f-statistics comparing across-category variance (e.g. horse vs cat) and variance within category (for photo-id)
f_stat_pixels = compute_f_stat(pixel_dists_frame)
f_stat_vgg19 = compute_f_stat(pixel_dists_frame_fc6)

plt.figure(figsize=(8,5))

# plot pdf of F-distribution with df1 = 11, df2 = 24
x = np.linspace(0, 7, 100)
plt.plot(x, f(11, 24).pdf(x), label=r'F-distribution, df$_1$ = 11, df$_2$= 24')

# add computed f-statistics from the pixel-level and VGG-19 fc6 data
plt.axvline(f_stat_pixels, color='orange', label='pixel-level features')
plt.axvline(f_stat_vgg19, color='green', label='VGG 19 features')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', prop={'size': 12})
plt.xlabel('F'), plt.ylabel('Density'), plt.suptitle('Between-class variability vs within-class (photo-cue) variability');

In [ ]:
f(11, 24).interval(.90)[1]

We see that between-category variability is large relative to within photo-id variability only in the VGG-19 features but not in pixel-level features, suggesting that category improves as a grouping variable when the grouping features are more abstract <br><br>

### RDM matrices: text vs photo condition

In [ ]:
between_condition_RDM(pixel_dists_frame, 'pixel-level', 'euclidean')

In [ ]:
between_condition_RDM(pixel_dists_frame, 'pixel-level', 'cosine')

In [ ]:
between_condition_RDM(pixel_dists_frame, 'pixel-level', 'correlation')

In [ ]:
between_condition_RDM(pixel_dists_frame_fc6, 'feature-level', 'euclidean')

In [ ]:
between_condition_RDM(pixel_dists_frame_fc6, 'feature-level', 'cosine')

In [ ]:
between_condition_RDM(pixel_dists_frame_fc6, 'feature-level', 'correlation')

### Are there any visible patterns among low-level data (detail metrics) between condition?

In [ ]:
# create a dataframe that allows us to compare the lowest level data with the pixel-level and fc6 features
low_level_frame = data.metadata.groupby(['category', 'condition', 'photoid'])[['numStrokes', 'activeSketchTime', 'totalInk']].aggregate(np.mean).reset_index()
lowfeatures_norm = preprocessing.scale(low_level_frame[['numStrokes', 'activeSketchTime', 'totalInk']].values)
lowfeatures_norm = [lowfeatures_norm[i] for i in range(len(lowfeatures_norm))]
low_level_frame = low_level_frame.assign(mean_feature = lowfeatures_norm)

In [ ]:
# as an interesting observation:
compute_f_stat(low_level_frame)

In [ ]:
between_condition_RDM(low_level_frame, 'low-level', 'euclidean')

In [ ]:
between_condition_RDM(low_level_frame, 'low-level', 'cosine')

In [ ]:
between_condition_RDM(low_level_frame, 'low-level', 'correlation')

## Inferential statistics: differences in effort in photo- and text-cue

#### Remember that we are conducting many t-tests here: apply bonferroni correction for the interpretation of significance levels

In [ ]:
# compute paired t-tests comparing photo/text condition for each basic level variable
for var in ['numStrokes', 'activeSketchTime', 'totalInk']:
    photodata = data.metadata[data.metadata.condition == 'photo'][var].values
    textdata = data.metadata[data.metadata.condition == 'text'][var].values
    
    print(f"Is mean {var} in photo vs text condition significantly different?:")
    
    # Are the variances approximately equal?
    varstats = stats.levene(photodata, textdata)
    print("Testing for equality of variance:")
    print(f"Levene test stat: {varstats[0]}. p-value: {varstats[1]}")
    if stats.levene(photodata, textdata)[1] < 0.05:
        welchtest = stats.ttest_ind(photodata, textdata, equal_var = False)
        print('The assumption for equality of variance is violated! Using Welch\'s t-test (two-sided), we get:')
        print(f'Welch\'s test stat: {welchtest[0]}. p-value: {welchtest[1]}\n')
    else:
        ttest = stats.ttest_ind(photodata, textdata)
        print('The assumption for equality of variance holds. Using student\'s t-test (two-sided), we get:')
        print(f'Student\'s t-test: {ttest[0]}. p-value: {ttest[1]}\n')